In [1]:
import os

project_folder = './pytorch-peds'
os.makedirs(project_folder, exist_ok=True)

Possibly helpful: [this link](https://github.com/drabastomek/GTC/blob/master/SJ_2020/workshop/1_Setup/Setup.ipynb), and this sample dockerfile from Jordan:

```
FROM mcr.microsoft.com/azureml/base-gpu:intelmpi2018.3-cuda9.0-cudnn7-ubuntu16.04

# Install Horovod, temporarily using CUDA stubsddd 
RUN ldconfig /usr/local/cuda/lib64/stubs && \     
# Install AzureML SDK     
pip install --no-cache-dir azureml-defaults && \     
# Install PyTorch     
pip install --no-cache-dir tensorflow==2.0.0b1 tensorflow-gpu==2.0.0b1 keras==2.0.8 matplotlib==3.0.3 seaborn==0.9.0 requests==2.21.0 bs4==0.0.1 imageio==2.5.0 sklearn pandas==0.24.2 numpy==1.16.2 hickle==3.4.3 && \     
# Install Horovod     
pip install --no-cache-dir horovod==0.13.5 && \     ldconfig
```

In [2]:
import shutil

shutil.copy('script.py', project_folder)

'./pytorch-peds/script.py'

In [3]:
#import sys
#sys.path.insert(0,'./')
# Install pycocotools
# !git clone https://github.com/cocodataset/cocoapi.git
# !ls
# %cd cocoapi/PythonAPI
# !ls
# !pip install cython
# !pip install numpy
# !python setup.py build_ext install

In [4]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.85


In [5]:
from azureml.telemetry import set_diagnostics_collection

set_diagnostics_collection(send_diagnostics=True)

Turning diagnostics collection on. 


In [6]:
from azureml.core.workspace import Workspace

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

Workspace name: gopalv-ws
Azure region: westus2
Subscription id: 15ae9cb6-95c1-483d-a0e3-b1a1a3b06324
Resource group: aifxdemo


In [7]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "gpu-cluster1"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', 
                                                           max_nodes=4)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

Found existing compute target.
{'currentNodeCount': 0, 'targetNodeCount': 0, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2020-02-26T06:11:31.534000+00:00', 'errors': None, 'creationTime': '2020-02-14T22:38:17.474022+00:00', 'modifiedTime': '2020-02-14T22:39:03.552424+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT120S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_NC6'}


In [8]:
import urllib.request
 
url = 'https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip'
 
urllib.request.urlretrieve(url, './test.zip')

('./test.zip', <http.client.HTTPMessage at 0x11d1a79d0>)

In [9]:
from zipfile import ZipFile

zip = ZipFile(file='./test.zip')
zip.extractall()

In [10]:
!ls PennFudanPed/

Annotation            PedMasks              readme.txt
PNGImages             added-object-list.txt


In [11]:
# get the default datastore
ds = ws.get_default_datastore()
print(ds.name, ds.datastore_type, ds.account_name, ds.container_name)

workspaceblobstore AzureBlob gopalvws3790775563 azureml-blobstore-e47496c6-9688-4277-a05b-ceb722514b9d


In [12]:
ds.upload('./PennFudanPed', target_path='data', overwrite=False)

Uploading an estimated of 512 files
Target already exists. Skipping upload for data/added-object-list.txt
Target already exists. Skipping upload for data/readme.txt
Target already exists. Skipping upload for data/PedMasks/PennPed00048_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00049_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00072_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00073_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00001_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00093_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00092_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00076_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00077_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00005_mask.png
Target already exists. Skipping upload for

Target already exists. Skipping upload for data/PedMasks/FudanPed00058_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00059_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00062_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00063_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00032_mask.png
Target already exists. Skipping upload for data/PedMasks/FudanPed00033_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00008_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00009_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00041_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00040_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00036_mask.png
Target already exists. Skipping upload for data/PedMasks/PennPed00037_mask.png
Target already exists. Skipping upload for data/

Target already exists. Skipping upload for data/Annotation/PennPed00065.txt
Target already exists. Skipping upload for data/Annotation/PennPed00071.txt
Target already exists. Skipping upload for data/Annotation/PennPed00067.txt
Target already exists. Skipping upload for data/Annotation/PennPed00073.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00074.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00060.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00048.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00049.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00061.txt
Target already exists. Skipping upload for data/Annotation/PennPed00072.txt
Target already exists. Skipping upload for data/Annotation/PennPed00066.txt
Target already exists. Skipping upload for data/Annotation/PennPed00089.txt
Target already exists. Skipping upload for data/Annotation/PennPed00062.txt
Target 

Target already exists. Skipping upload for data/Annotation/FudanPed00027.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00026.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00032.txt
Target already exists. Skipping upload for data/Annotation/PennPed00035.txt
Target already exists. Skipping upload for data/Annotation/PennPed00021.txt
Target already exists. Skipping upload for data/Annotation/PennPed00009.txt
Target already exists. Skipping upload for data/Annotation/PennPed00037.txt
Target already exists. Skipping upload for data/Annotation/PennPed00023.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00018.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00024.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00030.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00031.txt
Target already exists. Skipping upload for data/Annotation/FudanPed00025.txt
Targ

Target already exists. Skipping upload for data/PNGImages/PennPed00021.png
Target already exists. Skipping upload for data/PNGImages/PennPed00035.png
Target already exists. Skipping upload for data/PNGImages/PennPed00009.png
Target already exists. Skipping upload for data/PNGImages/PennPed00008.png
Target already exists. Skipping upload for data/PNGImages/PennPed00034.png
Target already exists. Skipping upload for data/PNGImages/PennPed00020.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00027.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00033.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00025.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00031.png
Target already exists. Skipping upload for data/PNGImages/FudanPed00019.png
Target already exists. Skipping upload for data/PNGImages/PennPed00036.png
Target already exists. Skipping upload for data/PNGImages/PennPed00022.png
Target already exist

$AZUREML_DATAREFERENCE_3604efa7da3c47ea9f9fbc0c464438aa

In [13]:
from azureml.core import Dataset
# create a FileDataset pointing to files in 'animals' folder and its subfolders recursively
datastore_paths = [(ds, 'data')]
penn_ds = Dataset.File.from_files(path=datastore_paths)
penn_ds.register(workspace=ws,
                 name='penn_ds',
                 description='Penn Fudan pedestrian data')

{
  "source": [
    "('workspaceblobstore', 'data')"
  ],
  "definition": [
    "GetDatastoreFiles"
  ],
  "registration": {
    "id": "1cd2dd50-ae6d-44f9-81e2-f0044c862efe",
    "name": "penn_ds",
    "version": 1,
    "description": "Penn Fudan pedestrian data",
    "workspace": "Workspace.create(name='gopalv-ws', subscription_id='15ae9cb6-95c1-483d-a0e3-b1a1a3b06324', resource_group='aifxdemo')"
  }
}

In [14]:
# dataset_name = 'penn_ds'

# # Get a dataset by name
# penn1_ds = Dataset.get_by_name(workspace=ws, name=dataset_name)

In [15]:
# from PIL import Image
# import os
# import glob


# with penn_ds.mount() as mount_context:
#     # list top level mounted files and folders in the dataset
#     imgs_path =(os.path.join(mount_context.mount_point, 'PNGImages'))
#     print(f'imgs_path is {imgs_path}')
#     print(list(sorted(os.listdir(imgs_path))))
#     Image.open(os.path.join(mount_context.mount_point, 'PNGImages/PennPed00036.png'))
# penn1_ds.as_named_input('test').as_mount()

In [16]:
# PennFudanDataset(penn_ds)

In [17]:
# !ls
# %cd ..
# !ls

In [18]:
!git clone https://github.com/pytorch/vision.git

!git checkout v0.3.0

%cd vision
!cp references/detection/utils.py ../
!cp references/detection/transforms.py ../
!cp references/detection/coco_eval.py ../
!cp references/detection/engine.py ../
!cp references/detection/coco_utils.py ../
%cd ..

fatal: destination path 'vision' already exists and is not an empty directory.
error: pathspec 'v0.3.0' did not match any file(s) known to git
/Users/gkv/MS/pytorch-object/vision
/Users/gkv/MS/pytorch-object


In [19]:
import shutil

os.listdir()

files_to_copy = ['utils', 'transforms', 'coco_eval', 'engine', 'coco_utils']
for file in files_to_copy:
    shutil.copy('./vision/references/detection/'+ file + '.py', project_folder)
shutil.copy('./conda-dependencies.yml', project_folder)


'./pytorch-peds/conda-dependencies.yml'

In [20]:
from azureml.core import Experiment

experiment_name = 'pytorch-peds'
experiment = Experiment(ws, name=experiment_name)

In [21]:
from azureml.core import Environment

my_env = Environment(name='maskr-docker')
my_env.docker.enabled = True
with open("dockerfiles/Dockerfile1", "r") as f:
    dockerfile_contents_of_your_base_image=f.read()
my_env.docker.base_dockerfile=dockerfile_contents_of_your_base_image 
my_env.docker.base_image = None # 'mcr.microsoft.com/azureml/base-gpu:openmpi3.1.2-cuda10.1-cudnn7-ubuntu18.04'
my_env.python.interpreter_path = '/opt/miniconda/bin/python'
my_env.python.user_managed_dependencies = True

# from azureml.core import Environment

# my_env = Environment.from_conda_specification(name = "myenv",
#                                              file_path = "conda-dependencies.yml")

In [22]:
# from azureml.train.estimator import Estimator

# my_est = Estimator(
#     source_directory=project_folder,
#     entry_script="script.py",
#     compute_target="local",
#     use_gpu=True,
#     conda_packages=['torchvision">=0.5.0"', 'torch', 'numpy', 'cython'])

In [23]:
from azureml.train.dnn import PyTorch
from azureml.core import ScriptRunConfig

# follow pattern from here: https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments#use-environments-for-training

# Add training script to run config
runconfig = ScriptRunConfig(source_directory=project_folder, script="script.py")

# Attach compute target to run config
runconfig.run_config.target = cluster_name#"local"


# Attach environment to run config
runconfig.run_config.environment = my_env

# Submit run 
run = experiment.submit(runconfig)
print(run)

Run(Experiment: pytorch-peds,
Id: pytorch-peds_1582723155_d65b2dc8,
Type: azureml.scriptrun,
Status: Starting)


In [24]:
# to get more details of your run
print(run.get_details())

{'runId': 'pytorch-peds_1582723155_d65b2dc8', 'target': 'gpu-cluster1', 'status': 'Starting', 'properties': {'_azureml.ComputeTargetType': 'amlcompute', 'ContentSnapshotId': '8c5d3450-024f-4be9-99ce-bdd25b9eaa2b', 'azureml.git.repository_uri': 'https://github.com/gvashishtha/pytorch-object.git', 'mlflow.source.git.repoURL': 'https://github.com/gvashishtha/pytorch-object.git', 'azureml.git.branch': 'docker', 'mlflow.source.git.branch': 'docker', 'azureml.git.commit': 'e8686c460e7df5f70febd76bb1915f17c615569a', 'mlflow.source.git.commit': 'e8686c460e7df5f70febd76bb1915f17c615569a', 'azureml.git.dirty': 'False'}, 'inputDatasets': [], 'runDefinition': {'script': 'script.py', 'useAbsolutePath': False, 'arguments': [], 'sourceDirectoryDataStore': None, 'framework': 'Python', 'communicator': 'None', 'target': 'gpu-cluster1', 'dataReferences': {}, 'data': {}, 'jobName': None, 'maxRunDurationSeconds': None, 'nodeCount': 1, 'environment': {'name': 'maskr-docker', 'version': 'Autosave_2020-02-26T

In [25]:
from azureml.widgets import RunDetails

RunDetails(run).show()
run.wait_for_completion(show_output=True)

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 's…

RunId: pytorch-peds_1582723155_d65b2dc8
Web View: https://ml.azure.com/experiments/pytorch-peds/runs/pytorch-peds_1582723155_d65b2dc8?wsid=/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourcegroups/aifxdemo/workspaces/gopalv-ws

Streaming azureml-logs/20_image_build_log.txt

2020/02/26 13:19:33 Downloading source code...
2020/02/26 13:19:34 Finished downloading source code
2020/02/26 13:19:34 Creating Docker network: acb_default_network, driver: 'bridge'
2020/02/26 13:19:35 Successfully set up Docker network: acb_default_network
2020/02/26 13:19:35 Setting up Docker configuration...
2020/02/26 13:19:35 Successfully set up Docker configuration
2020/02/26 13:19:35 Logging in to registry: gopalvwsdebb3b12.azurecr.io
2020/02/26 13:19:37 Successfully logged into gopalvwsdebb3b12.azurecr.io
2020/02/26 13:19:37 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2020/02/26 13:19:37 Scanning for dependencies...
2020/02/26 13:19:37 Su


idna-2.8             | 103 KB    |            |   0% 
idna-2.8             | 103 KB    | ########## | 100% 

openssl-1.1.1d       | 3.7 MB    |            |   0% 
openssl-1.1.1d       | 3.7 MB    | #######7   |  78% 
openssl-1.1.1d       | 3.7 MB    | ########## | 100% 

chardet-3.0.4        | 170 KB    |            |   0% 
chardet-3.0.4        | 170 KB    | ########## | 100% 

cryptography-2.8     | 618 KB    |            |   0% 
cryptography-2.8     | 618 KB    | ########6  |  86% 
cryptography-2.8     | 618 KB    | ########## | 100% 

zlib-1.2.11          | 120 KB    |            |   0% 
zlib-1.2.11          | 120 KB    | ########## | 100% 

pycosat-0.6.3        | 113 KB    |            |   0% 
pycosat-0.6.3        | 113 KB    | ########## | 100% 

pip-20.0.2           | 1.9 MB    |            |   0% 
pip-20.0.2           | 1.9 MB    | #######8   |  78% 
pip-20.0.2           | 1.9 MB    | #########7 |  98% 
pip-20.0.2           | 1.9 MB    | ########## | 100% 

_libgcc_mutex-0.1   


olefile-0.46         | 33 KB     |            |   0% 
olefile-0.46         | 33 KB     | ########## | 100% 

freetype-2.9.1       | 550 KB    |            |   0% 
freetype-2.9.1       | 550 KB    | ########## | 100% 

pillow-7.0.0         | 601 KB    |            |   0% 
pillow-7.0.0         | 601 KB    | ########## | 100% 

cudatoolkit-10.1.243 | 347.4 MB  |            |   0% 
cudatoolkit-10.1.243 | 347.4 MB  | 2          |   3% 
cudatoolkit-10.1.243 | 347.4 MB  | 6          |   6% 
cudatoolkit-10.1.243 | 347.4 MB  | 9          |  10% 
cudatoolkit-10.1.243 | 347.4 MB  | #3         |  13% 
cudatoolkit-10.1.243 | 347.4 MB  | #6         |  17% 
cudatoolkit-10.1.243 | 347.4 MB  | #9         |  20% 
cudatoolkit-10.1.243 | 347.4 MB  | ##3        |  23% 
cudatoolkit-10.1.243 | 347.4 MB  | ##6        |  27% 
cudatoolkit-10.1.243 | 347.4 MB  | ###        |  30% 
cudatoolkit-10.1.243 | 347.4 MB  | ###3       |  33% 
cudatoolkit-10.1.243 | 347.4 MB  | ###6       |  36% 
cudatoolkit-10.1.243 | 3

 ---> Running in 2f0b39abb3c9
  Created wheel for json-logging-py: filename=json_logging_py-0.2-py3-none-any.whl size=3923 sha256=42113b0eb9f6761a0c6df226d11d6cfa6435d740c46850f10a2db5d58b5ee9d2
  Stored in directory: /root/.cache/pip/wheels/e9/d6/70/7491901d808e74dd9238e4a91658ba108e4b5939b55327e6fb
  Created wheel for ruamel.yaml: filename=ruamel.yaml-0.15.89-cp38-cp38-linux_x86_64.whl size=672013 sha256=144d2db817c6d418fe0428870291b36a177cc7a5eb6c0f9d6af94891e31024b3
  Stored in directory: /root/.cache/pip/wheels/ff/6d/69/6a3ed44734986d96f09da482c41ecdce74a40fbe10d605bc9f
  Created wheel for liac-arff: filename=liac_arff-2.4.0-py3-none-any.whl size=13333 sha256=c0c378c670dc2d969e8f6fb8d69759cb42f96d947cb1471c3489e04cd10ac4e1
  Stored in directory: /root/.cache/pip/wheels/b1/a5/a6/e2fcffce8959ffa92c16d8ca4ab632299d73c18470b112263c
  Created wheel for dill: filename=dill-0.3.1.1-py3-none-any.whl size=78530 sha256=603fb0bb5e9dac659d4b46ec3ebeffedcc5cfc1906a64e7cfcc9c2b7e1ecafab
  Store

Removing intermediate container 2f0b39abb3c9
 ---> 2b0c08d2a87a
Step 8/15 : USER root
 ---> Running in 0e540b2b3bb0
Removing intermediate container 0e540b2b3bb0
 ---> 5099cc086815
Step 9/15 : RUN mkdir -p $HOME/.cache
 ---> Running in 9337b132bd2e
Removing intermediate container 9337b132bd2e
 ---> bda79d68e2fe
Step 10/15 : WORKDIR /
 ---> Running in 6f84d4fe1bd3
Removing intermediate container 6f84d4fe1bd3
 ---> bd508160240b
Step 11/15 : COPY azureml-environment-setup/99brokenproxy /etc/apt/apt.conf.d/
 ---> f81604e379b8
Step 12/15 : COPY azureml-environment-setup/spark_cache.py azureml-environment-setup/log4j.properties /azureml-environment-setup/
 ---> d5d8d6a74ee3
Step 13/15 : RUN if [ $SPARK_HOME ]; then /bin/bash -c '$SPARK_HOME/bin/spark-submit  /azureml-environment-setup/spark_cache.py'; fi
 ---> Running in e0d9a7244eb3
Removing intermediate container e0d9a7244eb3
 ---> a67dae010328
Step 14/15 : ENV AZUREML_ENVIRONMENT_IMAGE True
 ---> Running in 5c8a52950a94
Removing intermedia

WARNING - Retrying (Retry(total=2, connect=3, read=2, redirect=None, status=None)) after connection broken by 'ProtocolError('Connection aborted.', OSError("(65, 'EHOSTUNREACH')"))': /azureml/ExperimentRun/dcid.pytorch-peds_1582723155_d65b2dc8/azureml-logs/75_job_post-tvmps_832258b0741f11e2adf8cf7f00d4df0f1ea49070eb01a3096f0e4162e0fa6b30_d.txt?sv=2019-02-02&sr=b&sig=I6U3eS1rW9tDOqI8S5S%2FlYteaSM%2FiSE%2Bu5TXnAG5KbI%3D&st=2020-02-26T13%3A34%3A25Z&se=2020-02-26T21%3A44%3A25Z&sp=r


Starting job release. Current time:2020-02-26T13:38:45.094429
Logging experiment finalizing status in history service.
Starting the daemon thread to refresh tokens in background for process with pid = 158
Job release is complete. Current time:2020-02-26T13:38:46.408723

Execution Summary
RunId: pytorch-peds_1582723155_d65b2dc8
Web View: https://ml.azure.com/experiments/pytorch-peds/runs/pytorch-peds_1582723155_d65b2dc8?wsid=/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourcegroups/aifxdemo/workspaces/gopalv-ws

Warnings:
{
  "error": {
    "code": "UserError",
    "message": "AzureMLCompute job failed.\nJobFailed: Submitted script failed with a non-zero exit code; see the driver log file for details.",
    "detailsUri": null,
    "target": null,
    "details": [],
    "innerError": null,
    "debugInfo": null
  },
  "correlation": {
    "operation": null,
    "request": "f2112ef63e404b5c"
  },
  "environment": "westus2",
  "location": "westus2",
  "time": "2020-02-26T13:38:58.

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "User program failed with ImportError: Could not import package \"azureml-dataprep\". Please ensure it is installed by running: pip install \"azureml-dataprep[fuse,pandas]\"",
        "detailsUri": "https://aka.ms/azureml-known-errors",
        "details": [],
        "debugInfo": {
            "type": "ImportError",
            "message": "Could not import package \"azureml-dataprep\". Please ensure it is installed by running: pip install \"azureml-dataprep[fuse,pandas]\"",
            "stackTrace": "  File \"/mnt/batch/tasks/shared/LS_root/jobs/gopalv-ws/azureml/pytorch-peds_1582723155_d65b2dc8/mounts/workspaceblobstore/azureml/pytorch-peds_1582723155_d65b2dc8/azureml-setup/context_manager_injector.py\", line 127, in execute_with_context\n    runpy.run_path(sys.argv[0], globals(), run_name=\"__main__\")\n  File \"/opt/miniconda/lib/python3.8/run

In [90]:
from azureml.train.estimator import Estimator

script_params = {
    '--num_epochs': 10,
    '--output_dir': './outputs'
}

estimator = Estimator(source_directory=project_folder, 
                      script_params=script_params,
                      compute_target=compute_target,
                      entry_script='script.py',
                      use_gpu=True,
                      conda_dependencies_file='conda-dependencies.yml')

In [91]:
run = experiment.submit(estimator)
print(run.get_details())

TrainingException: TrainingException:
	Message: {
    "error_details": {
        "correlation": {
            "operation": "04acdb6530c8a64a9a441734c145be21",
            "request": "0e0fa31e21b64f25"
        },
        "environment": "westus2",
        "error": {
            "code": "UserError",
            "debugInfo": {
                "innerException": {
                    "errorResponse": {
                        "correlation": {
                            "operation": "04acdb6530c8a64a9a441734c145be21",
                            "request": "49e641e39e5594c9"
                        },
                        "environment": "westus2",
                        "error": {
                            "code": "ValidationError",
                            "debugInfo": {
                                "message": "Python interpreter version must be specified in the Conda file.",
                                "stackTrace": "   at Microsoft.MachineLearning.EnvironmentManagement.Ser

In [70]:
from azureml.widgets import RunDetails

RunDetails(run).show()
run.wait_for_completion(show_output=True)

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': True, 'log_level': 'INFO', 's…

RunId: pytorch-peds_1582507355_50a4f6df
Web View: https://ml.azure.com/experiments/pytorch-peds/runs/pytorch-peds_1582507355_50a4f6df?wsid=/subscriptions/15ae9cb6-95c1-483d-a0e3-b1a1a3b06324/resourcegroups/aifxdemo/workspaces/gopalv-ws

Streaming azureml-logs/20_image_build_log.txt

2020/02/24 01:22:52 Downloading source code...
2020/02/24 01:22:53 Finished downloading source code
2020/02/24 01:22:54 Creating Docker network: acb_default_network, driver: 'bridge'
2020/02/24 01:22:54 Successfully set up Docker network: acb_default_network
2020/02/24 01:22:54 Setting up Docker configuration...
2020/02/24 01:22:55 Successfully set up Docker configuration
2020/02/24 01:22:55 Logging in to registry: gopalvwsdebb3b12.azurecr.io
2020/02/24 01:22:56 Successfully logged into gopalvwsdebb3b12.azurecr.io
2020/02/24 01:22:56 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2020/02/24 01:22:56 Scanning for dependencies...
2020/02/24 01:22:57 Su

Executing transaction: ...working... done
ERROR: Could not find a version that satisfies the requirement cocoapi (from -r /azureml-environment-setup/condaenv.9ty031gd.requirements.txt (line 2)) (from versions: none)
ERROR: No matching distribution found for cocoapi (from -r /azureml-environment-setup/condaenv.9ty031gd.requirements.txt (line 2))


CondaValueError: pip returned an error

The command '/bin/sh -c ldconfig /usr/local/cuda/lib64/stubs && conda env create -p /azureml-envs/azureml_a45a0b01d0acbd550b290c3aa915e399 -f azureml-environment-setup/mutated_conda_dependencies.yml && rm -rf "$HOME/.cache/pip" && conda clean -aqy && CONDA_ROOT_DIR=$(conda info --root) && rm -rf "$CONDA_ROOT_DIR/pkgs" && find "$CONDA_ROOT_DIR" -type d -name __pycache__ -exec rm -rf {} + && ldconfig' returned a non-zero code: 1
2020/02/24 01:25:05 Container failed during run: acb_step_0. No retries remaining.
failed to run step ID: acb_step_0: exit status 1

Run ID: ccj failed after 2m13s. Error: failed d

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Image build failed. For more details, check log file azureml-logs/20_image_build_log.txt",
        "details": []
    },
    "correlation": {
        "operation": null,
        "request": "f778c1b7c41ef175"
    },
    "environment": "westus2",
    "location": "westus2",
    "time": "2020-02-24T01:25:08.642942Z"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"Image build failed. For more details, check log file azureml-logs/20_image_build_log.txt\",\n        \"details\": []\n    },\n    \"correlation\": {\n        \"operation\": null,\n        \"request\": \"f778c1b7c41ef175\"\n    },\n    \"environment\": \"westus2\",\n    \"location\": \"westus2\",\n    \"time\": \"2020-02-24T01:25:08.642942Z\"\n}"
    }
}

In [14]:
import os
import numpy as np
import torch
import torch.utils.data
from PIL import Image


class PennFudanDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, transforms=None):
        self.dataset = dataset
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        with self.dataset.mount() as mount_context:
            self.imgs = list(sorted(os.listdir(os.path.join(mount_context.mount_point, "PNGImages"))))
            self.masks = list(sorted(os.listdir(os.path.join(mount_context.mount_point, "PedMasks"))))

    def __getitem__(self, idx):
        # load images ad masks
        with self.dataset.mount() as mount_context:
            img_path = os.path.join(mount_context.mount_point, "PNGImages", self.imgs[idx])
            mask_path = os.path.join(mount_context.mount_point, "PedMasks", self.masks[idx])
            img = Image.open(img_path).convert("RGB")
            # note that we haven't converted the mask to RGB,
            # because each color corresponds to a different instance
            # with 0 being background
            mask = Image.open(mask_path)

            mask = np.array(mask)
            # instances are encoded as different colors
            obj_ids = np.unique(mask)
            # first id is the background, so remove it
            obj_ids = obj_ids[1:]

            # split the color-encoded mask into a set
            # of binary masks
            masks = mask == obj_ids[:, None, None]

            # get bounding box coordinates for each mask
            num_objs = len(obj_ids)
            boxes = []
            for i in range(num_objs):
                pos = np.where(masks[i])
                xmin = np.min(pos[1])
                xmax = np.max(pos[1])
                ymin = np.min(pos[0])
                ymax = np.max(pos[0])
                boxes.append([xmin, ymin, xmax, ymax])

            boxes = torch.as_tensor(boxes, dtype=torch.float32)
            # there is only one class
            labels = torch.ones((num_objs,), dtype=torch.int64)
            masks = torch.as_tensor(masks, dtype=torch.uint8)

            image_id = torch.tensor([idx])
            area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
            # suppose all instances are not crowd
            iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

            target = {}
            target["boxes"] = boxes
            target["labels"] = labels
            target["masks"] = masks
            target["image_id"] = image_id
            target["area"] = area
            target["iscrowd"] = iscrowd

            if self.transforms is not None:
                img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

In [16]:
!pip install --upgrade torchvision

Requirement already up-to-date: torchvision in /Users/gkv/miniconda3/envs/azureml/lib/python3.8/site-packages (0.5.0)


In [27]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

      
def get_instance_segmentation_model(num_classes):
    # load an instance segmentation model pre-trained on COCO
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    # get the number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    # now get the number of input features for the mask classifier
    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256
    # and replace the mask predictor with a new one
    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [19]:
import sys
sys.path.insert(0,'./')

In [20]:
from platform import python_version

print(python_version())
help("modules")

3.8.1

Please wait a moment while I gather a list of all available modules...



/Users/gkv/miniconda3/envs/azureml/lib/python3.8/site-packages/IPython/kernel/__init__.py:12: ShimWarning: The `IPython.kernel` package has been deprecated since IPython 4.0.You should import from ipykernel or jupyter_client instead.
  warn("The `IPython.kernel` package has been deprecated since IPython 4.0."
/Users/gkv/miniconda3/envs/azureml/lib/python3.8/pkgutil.py:107: VisibleDeprecationWarning: zmq.eventloop.minitornado is deprecated in pyzmq 14.0 and will be removed.
    Install tornado itself to use zmq with the tornado IOLoop.
    
  yield from walk_packages(path, info.name+'.', onerror)


Cython              asynchat            inspect             random
IPython             asyncio             io                  re
OpenSSL             asyncore            ipaddress           readline
PIL                 atexit              ipykernel           reprlib
PyQt5               attr                ipykernel_launcher  requests
__future__          audioop             ipython_genutils    requests_oauthlib
_abc                automl              ipywidgets          resource
_ast                autoreload          isodate             rlcompleter
_asyncio            azureml             itertools           rmagic
_bisect             backcall            jaraco              runpy
_blake2             backports           jedi                sched
_bootlocale         base64              jeepney             secrets
_bz2                bdb                 jinja2              secretstorage
_cffi_backend       binascii            jmespath            select
_codecs             binhex           

/Users/gkv/miniconda3/envs/azureml/lib/python3.8/pkgutil.py:92: MatplotlibDeprecationWarning: 
The mpl_toolkits.axes_grid module was deprecated in Matplotlib 2.1 and will be removed two minor releases later. Use mpl_toolkits.axes_grid1 and mpl_toolkits.axisartist, which provide the same functionality instead.
  __import__(info.name)


In [28]:
# from engine import train_one_epoch, evaluate
# import utils
# import transforms as T

# def get_transform(train):
#     transforms = []
#     # converts the image, a PIL image, into a PyTorch Tensor
#     transforms.append(T.ToTensor())
#     if train:
#         # during training, randomly flip the training images
#         # and ground-truth for data augmentation
#         transforms.append(T.RandomHorizontalFlip(0.5))
#     return T.Compose(transforms)


In [33]:
# # use our dataset and defined transformations
# dataset = PennFudanDataset(penn_ds, get_transform(train=True))
# dataset_test = PennFudanDataset(penn_ds, get_transform(train=False))

# # split the dataset in train and test set
# torch.manual_seed(1)
# indices = torch.randperm(len(dataset)).tolist()
# dataset = torch.utils.data.Subset(dataset, indices[:-50])
# dataset_test = torch.utils.data.Subset(dataset_test, indices[-50:])

# # define training and validation data loaders
# data_loader = torch.utils.data.DataLoader(
#     dataset, batch_size=2, shuffle=True, num_workers=4,
#     collate_fn=utils.collate_fn)

# data_loader_test = torch.utils.data.DataLoader(
#     dataset_test, batch_size=1, shuffle=False, num_workers=4,
#     collate_fn=utils.collate_fn)

In [34]:
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# # our dataset has two classes only - background and person
# num_classes = 2

# # get the model using our helper function
# model = get_instance_segmentation_model(num_classes)
# # move model to the right device
# model.to(device)

# # construct an optimizer
# params = [p for p in model.parameters() if p.requires_grad]
# optimizer = torch.optim.SGD(params, lr=0.005,
#                             momentum=0.9, weight_decay=0.0005)

# # and a learning rate scheduler which decreases the learning rate by
# # 10x every 3 epochs
# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer,
#                                                step_size=3,
#                                                gamma=0.1)

In [1]:
# num_epochs = 10

# for epoch in range(num_epochs):
#     # train for one epoch, printing every 10 iterations
#     train_one_epoch(model, optimizer, data_loader, device, epoch, print_freq=10)
#     # update the learning rate
#     lr_scheduler.step()
#     # evaluate on the test dataset
#     evaluate(model, data_loader_test, device=device)

NameError: name 'train_one_epoch' is not defined

In [ ]:
# # pick one image from the test set
# img, _ = dataset_test[0]
# # put the model in evaluation mode
# model.eval()
# with torch.no_grad():
#     prediction = model([img.to(device)])

In [ ]:
# Image.fromarray(img.mul(255).permute(1, 2, 0).byte().numpy())